In [2]:
# ==============================
# 🔹 Install Dependencies
# ==============================
!pip -q install langchain langchain-community faiss-cpu sentence-transformers gradio pandas nltk

import os, time, shutil, subprocess
import pandas as pd

# ==============================
# 🔹 Setup Ollama (Linux/Colab)
# ==============================
def run(cmd):
    print("> ", cmd)
    return subprocess.call(cmd, shell=True)

if shutil.which("ollama") is None:
    run("curl -fsSL https://ollama.com/install.sh | sh")

# Start Ollama server
_ = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)

# Pull required models
run("ollama pull llama3.1:8b")
run("ollama pull nomic-embed-text")

import nltk
nltk.download('punkt', quiet=True)
print("✅ Setup complete")


# ==============================
# 🔹 Config
# ==============================
COLLEGE_CSV = "hyderabad_engineering_colleges.csv"
QA_CSV = "hyd_college_faq_extended.csv"
EMBED_MODEL = "nomic-embed-text"
GEN_MODEL = "llama3.1:8b"
TOP_K = 10
TEMPERATURE = 0.3


# ==============================
# 🔹 Load CSVs
# ==============================
assert os.path.exists(COLLEGE_CSV), "College CSV not found!"
assert os.path.exists(QA_CSV), "FAQ CSV not found!"

df = pd.read_csv(COLLEGE_CSV).fillna("")
faq_df = pd.read_csv(QA_CSV).fillna("")
print("Colleges:", len(df), "| FAQs:", len(faq_df))


# ==============================
# 🔹 Column Normalization
# ==============================
CAND_COLS = {
    "name": ["college_name","College_Name","Name","College","Institute","College Name"],
    "area": ["area","Area","Location","location","Address","Locality"],
    "affiliation": ["affiliation","Affiliation","University"],
    "accr": ["approved_accreditations","Accreditations","accreditations","NAAC/NBA","NAAC","NBA"],
    "branches": ["branches","Branches","Courses","Specializations","Programs"],
    "fees": ["btech_fees_inr","Fees","Tuition","Annual Fee","Fee","Fee(₹)","B.Tech Fee","Annual Fees (INR)"],
    "website": ["website","Website","URL"],
    "placements": ["Placements","placements","Highest Package","Average Package","Placement Stats"],
    "cutoff": ["Cutoff","cutoff","TS EAMCET Cutoff","JEE Cutoff"],
    "rank": ["Rank","NIRF Rank","nirf_rank","College Rank"],
    "desc": ["Description","Summary","Overview"],
}

def pick(r, keys):
    for k in keys:
        if k in r and str(r[k]).strip():
            return str(r[k])
    return ""

def row_to_doc(row):
    r = row.to_dict()
    parts = []
    parts.append("College: " + pick(r, CAND_COLS["name"]))
    if (ar := pick(r, CAND_COLS["area"])): parts.append("Area: " + ar)
    if (af := pick(r, CAND_COLS["affiliation"])): parts.append("Affiliation: " + af)
    if (ac := pick(r, CAND_COLS["accr"])): parts.append("Accreditations: " + ac)
    if (br := pick(r, CAND_COLS["branches"])): parts.append("Branches: " + br)
    if (fe := pick(r, CAND_COLS["fees"])): parts.append("Fees(₹): " + fe)
    if (cu := pick(r, CAND_COLS["cutoff"])): parts.append("Cutoff: " + cu)
    if (pl := pick(r, CAND_COLS["placements"])): parts.append("Placements: " + pl)
    if (rk := pick(r, CAND_COLS["rank"])): parts.append("Rank: " + rk)
    if (ds := pick(r, CAND_COLS["desc"])): parts.append("About: " + ds)
    if (we := pick(r, CAND_COLS["website"])): parts.append("Website: " + we)
    return " | ".join(parts)

docs_text = [row_to_doc(row) for _, row in df.iterrows()]
faq_texts = [f"Q: {row['prompt']} | A: {row['answer']}" for _, row in faq_df.iterrows()]
all_docs = docs_text + faq_texts
print("📄 Documents prepared:", len(all_docs))


# ==============================
# 🔹 Embeddings & FAISS
# ==============================
from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

try:
    emb = OllamaEmbeddings(model=EMBED_MODEL)
    _ = emb.embed_query("hi")
    print("✅ Using OllamaEmbeddings:", EMBED_MODEL)
except Exception as e:
    print("⚠️ Ollama failed, using HuggingFace:", e)
    emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vs = FAISS.from_texts(all_docs, emb)
vs.save_local("faiss_hyd_colleges")
print("✅ Vector index built & saved")


# ==============================
# 🔹 RAG Chain
# ==============================
from langchain_community.chat_models import ChatOllama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

vs = FAISS.load_local("faiss_hyd_colleges", emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(search_kwargs={"k": TOP_K})

system_prompt = (
    "You are a friendly Hyderabad B.Tech admissions advisor.\n"
    "- If an answer is not found in the knowledge base, say 'Not available in database. Please check official college websites.'\n"
    "- Never invent colleges or info outside the retrieved data.\n"
    "- Answer naturally & conversationally.\n"
    "- Use bullet points for multi-item answers.\n"
    "- Remind that fees, cutoffs, and placements vary yearly.\n"
)

qa_prompt = PromptTemplate.from_template(
    "{system_prompt}\nChat history:\n{chat_history}\n\nContext:\n{context}\n\nUser: {question}\nAssistant:"
)

from langchain.callbacks.base import BaseCallbackHandler

# Streaming handler (optional: shows tokens in console)
class StreamHandler(BaseCallbackHandler):
    def __init__(self):
        self.buffer = ""
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)  # prints tokens live
        self.buffer += token

llm = ChatOllama(
    model=GEN_MODEL,
    temperature=TEMPERATURE,
    streaming=True,
    callbacks=[StreamHandler()]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": qa_prompt.partial(system_prompt=system_prompt)},
)

print("✅ RAG chain ready")


# ==============================
# 🔹 Gradio Chat UI
# ==============================
import gradio as gr

# 🔹 Streaming chat function
def chat_fn(message, history):
    response = ""
    try:
        # Build context manually (instead of qa chain)
        chat_history = "\n".join([f"User: {u}\nAssistant: {a}" for u, a in history])
        docs = retriever.get_relevant_documents(message)
        context = "\n".join([d.page_content for d in docs])
        full_prompt = qa_prompt.format(
            system_prompt=system_prompt,
            chat_history=chat_history,
            context=context,
            question=message,
        )

        # 🔹 Stream raw LLM tokens
        for chunk in llm.stream(full_prompt):
            token = chunk.content
            response += token
            yield response

        memory.chat_memory.add_user_message(message)
        memory.chat_memory.add_ai_message(response)

    except Exception as e:
        yield f"⚠️ Error: {str(e)}"


# 🔹 Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("<h2 style='text-align:center;'>🎓 Engg Assist - Hyderabad B.Tech Chatbot 🤖</h2>")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask about colleges, fees, cutoffs, placements...", autofocus=True)
    clear = gr.Button("Clear Chat")

    # ✅ Respond function with generator
    def respond(user_msg, chat_history):
        chat_history.append((user_msg, ""))   # reserve bot reply slot
        for partial in chat_fn(user_msg, chat_history):
            chat_history[-1] = (user_msg, partial)
            yield chat_history, ""   # stream updates to UI

    # Bind events
    msg.submit(respond, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

# 🔹 Launch
demo.launch(share=True)



>  ollama pull llama3.1:8b
>  ollama pull nomic-embed-text
✅ Setup complete
Colleges: 42 | FAQs: 236
📄 Documents prepared: 278
✅ Using OllamaEmbeddings: nomic-embed-text
✅ Vector index built & saved
✅ RAG chain ready


/tmp/ipython-input-942073553.py:207: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f0b07a659c1bcba684.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
